In [4]:
!pip show transformers || pip install transformers==4.41.2
!pip show trl || pip install trl

In [5]:
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import notebook_login
from trl import SFTTrainer, SFTConfig

In [6]:
base_path = "/content/drive/My Drive/yl_tez"

In [3]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
model_adi = "bestmodel"
model_yolu = os.path.join(base_path, model_adi)
repo_name = "ytu_doktor_gpt2-medium"
commit_message = "eyvallah"

In [8]:
notebook_login()

In [17]:
training_args = SFTConfig(
    output_dir=model_yolu,  # Modelin kaydedileceği dizin
    report_to="none",  # Raporlama aracı
    dataloader_drop_last=True,  # Son batch'i bırak
    per_device_eval_batch_size=8,  # Değerlendirme batch boyutu
    per_device_train_batch_size=8, # Her cihaz başına eğitim batch boyutu
    save_total_limit=3,  # En fazla 3 kontrol noktası sakla,,
    save_steps=2000,  # Kaydetme adımı, save_strategy="epoch" için gerek yok
    eval_steps=2000,  # Test etme adımı eval_strategy="epoch" için gerek yok
    logging_steps=2000,  # Loglama adımı logging_strategy="epoch" için gerek yok steps için var
    eval_strategy="steps",  # Değerlendirme stratejisi adım bazında
    save_strategy="steps",  # Kaydetme stratejisi adım bazında
    logging_strategy="steps",
    num_train_epochs=20,  # Eğitim epoch sayısı,
    gradient_checkpointing=False,  # Gradient checkpointing
    gradient_accumulation_steps=8,  # Gradient accumulation steps,
    fp16=False,  # 16-bit float training
    push_to_hub=True,  # Hugging Face Hub'a kaydet
    hub_model_id=repo_name,  # Hugging Face Hub'a kaydetme
)

In [18]:
trainer = SFTTrainer(
        model_yolu,
        args=training_args,
    )

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:174: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is

In [20]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kayrab/ytu_doktor_gpt2-medium/commit/41eae8b0bcc1b18dfd7a6a61c563821d849870e1', commit_message='End of training', commit_description='', oid='41eae8b0bcc1b18dfd7a6a61c563821d849870e1', pr_url=None, pr_revision=None, pr_num=None)